In [12]:
# Plot LFP effects for stimulation on the different channels
# Select 2-3 stimulation and recording channels
# Plot LFPs, PSDs, STFT, and cross-channel coherence
# Pre-stim only

#Import Libraries
# | echo: false
# | warning: false
%run C:/Users/27707/Documents/jhu_master/lab/importrhsutilities.py
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.extractors as sse
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.curation as scur
import spikeinterface.widgets as sw

from probeinterface import Probe, ProbeGroup
from probeinterface import generate_linear_probe, generate_multi_shank
from probeinterface import combine_probes
from probeinterface.plotting import plot_probe

import datetime
from uuid import uuid4

import numpy as np
from dateutil.tz import tzlocal

from pynwb import NWBHDF5IO, NWBFile
from pynwb.ecephys import LFP, ElectricalSeries
from pprint import pprint
from scipy import signal

In [13]:
recording_names = ['BO3_O5W_Sti_Day1_5uA_m.nwb',
                   'BO3_O5W_Sti_Day1_10uA_m.nwb',
                   'BO3_O5W_Sti_Day1_20uA_m.nwb',
                   'BO3_O5W_Sti_Day1_30uA_m.nwb',
                   'BO3_O5W_Sti_Day1_40uA_m.nwb',
                   'BO3_O5W_Sti_Day1_50uA_m.nwb',
                   'BO3_O5W_Sti_Day1_60uA_m.nwb'
                   ]

amps = ['5ua','10ua','20ua',
        '30ua','40ua','50ua','60ua']
intan_path = 'C:/Users/27707/Documents/jhu_master/LFP_analysis/2024-07-26_DG_3elec/Experimental Set_Final Round_Batch 27/5 Week Old Organoid/Stimulation Day1/'
intan_folders = ['5uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_5uA_233uS_240625_161619/merged/',
                 '10uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_10uA_233uS_240625_162505/merged/',
                 '20uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_20uA_233uS_240625_163352/merged/',
                 '30uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_30uA_233uS_240625_164235/merged/',
                 '40uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_40uA_233uS_240625_165135/merged/',
                 '50uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_50uA_233uS_240625_170022/merged/',
                 '60uA_233.3uS_Biphasic Pulse/BO3_O5W_Stimulation_Day1_60uA_233uS_240625_170906/merged/'
                 ]

recording_names1 = ['BO11_O7W_Sti_Day4_5uA_m.nwb',
                   'BO11_O7W_Sti_Day4_10uA_m.nwb',
                   'BO11_O7W_Sti_Day4_20uA_m.nwb',
                   'BO11_O7W_Sti_Day4_30uA_m.nwb',
                   'BO11_O7W_Sti_Day4_40uA_m.nwb',
                   'BO11_O7W_Sti_Day4_50uA_m.nwb',
                   'BO11_O7W_Sti_Day4_60uA_m.nwb'
                   ]

amps1 = ['5ua','10ua','20ua',
        '30ua','40ua','50ua','60ua']
intan_path1 = 'C:/Users/27707/Documents/jhu_master/LFP_analysis/2024-07-26_DG_3elec/Experimental Set_Final Round_Batch 27/7 Week Old Organoid/Stimulation Day1/'
intan_folders1 = ['5uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_5uA_240708_144920/merged/',
                 '10uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_10uA_240708_145755/merged/',
                 '20uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_20uA_240708_150743/merged/',
                 '30uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_30uA_240708_151657/merged/',
                 '40uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_40uA_240708_152646/merged/',
                 '50uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_50uA_240708_153556/merged/',
                 '60uA_233.3uS_Biphasic Pulse/BO11_O7W_Stimulation_Day4_60uA_240708_154509/merged/'
                 ]

In [14]:
def all_intan_data(channel_names, signal_data_name,results):
    dfs = []
    i = 0
    for res in results:
        #save times for later processing
        dict_data = {}
        dict_data['time'] = res['t']

        #add data for each channel to master df
        j = 0
        for chan in channel_names:
            channel_found, signal_type, signal_index = find_channel_in_header(chan, res)
            dict_data[chan] = res[signal_data_name][signal_index,:]/1000 # data is in mV but NWB electrical series type expects V
            j += 1
        df = pd.DataFrame.from_dict(dict_data)
        dfs.append(df)
        i+=1
    rec_data = pd.concat([dfs[i] for i in range(len(dfs))], axis=0)
    rec_data = rec_data.set_index('time')
    return rec_data

def pull_files(recording_name,fpath='/media/t7/surpass/electrophysiology/Gracias Data/ExperimentalSet_Round2/'):
    '''
    input:
    day - int - day number
    recording_name - string - string name of recording folder
    '''
    filepath = fpath + recording_name + '/*.rhs'
    print(filepath)
    fnames = glob.glob(filepath)
    print(fnames)
    results = 0
    for i in np.arange(len(fnames)):
        res, data_present = load_file(fnames[i])
        if i == 0:
            results = [res]
        else:
            results.append(res)
    return results    

In [ ]:
stim_times_array = []
for i in range(len(recording_names)):
    BO7_res = pull_files(intan_folders[i], intan_path)
    stim_names_BO7 = ['STIM_A-000','STIM_A-002','STIM_A-004']
    stim_BO7 = all_intan_data(stim_names_BO7,'stim_data',BO7_res)
    stim_times = stim_BO7.loc[(stim_BO7['STIM_A-000'] != 0) | (stim_BO7['STIM_A-002'] != 0) | (stim_BO7['STIM_A-004'] != 0)].index
    stim_times_array.append(stim_times.to_numpy())

In [ ]:
stim_start_array = []
stim_end_array = []
for stim in stim_times_array:
    stim = np.insert(stim,0,0)
    stim_times_diff = np.diff(stim)
    stim_start = stim[np.where(stim_times_diff > 1)[0]+1]
    stim_end = stim[np.where(stim_times_diff > 1)[0][1:]]
    stim_end = np.concatenate((stim_end,[stim[-1]]))
    print(stim_start)
    print(stim_end)
    stim_start_array.append(stim_start)
    stim_end_array.append(stim_end)

In [ ]:
stim_start_array[2][1]

In [18]:
def gen_probe():
    north = generate_linear_probe(num_elec=1)
    north.rotate(180)
    north.set_contacts(positions=[[0,75]])


    east = generate_linear_probe(num_elec=1)
    east.rotate(90)
    east.set_contacts(positions=[[75,0]])

    west = generate_linear_probe(num_elec=1)
    west.rotate(-90)
    west.set_contacts(positions=[[-75,0]])

    multi_shank = combine_probes([west, east, north])
    plot_probe(multi_shank)
    plt.show()
    multi_shank.set_device_channel_indices([0,1,2])
    return multi_shank

def filterbank(recording):
    recording_delta = spre.bandpass_filter(recording, freq_min=1, freq_max=4) #delta- moutri paper
    recording_100_200 = spre.bandpass_filter(recording, freq_min=100, freq_max=200) #'gamma'- moutri paper
    recording_200_400 = spre.bandpass_filter(recording, freq_min=200, freq_max=400) #'gamma'- moutri paper
    recording_gamma = spre.bandpass_filter(recording, freq_min=70, freq_max=110) #'ecog high gamma
    return recording_delta, recording_100_200, recording_200_400, recording_gamma

def downsample_lfp_mua(recording):
    recording_lfp = spre.bandpass_filter(recording, freq_min=1, freq_max=400)
    recording_lfp = spre.resample(recording_lfp, 1000)
    recording_mua = spre.resample(spre.rectify(recording), 1000)
    return recording_lfp, recording_mua

def load_data(rec_name):
    recording = se.read_nwb_recording(rec_name)
    multi_shank = gen_probe()
    recording = recording.set_probe(multi_shank)
    return recording 

In [ ]:

num_samples = []
num_channels = []
multirecording = []
multirecording_f = []
for i in range(len(recording_names)):
    recording_names = [folder + 'BO3_O5W_Sti_Day1_' + amp.split('u')[0] + 'uA_m.nwb' 
                       for folder, amp in zip(intan_folders, amps)]
    rec_names = intan_path + recording_names[i]
    day_recording = load_data(rec_names)
    [num_pre_samples, num_pre_channels] = day_recording.get_traces().shape # Num samples, num channels
    num_samples.append(num_pre_samples)
    num_channels.append(num_pre_channels)
    print('Presample:' + str(num_pre_samples))

    # Case 2: the sorter DOES NOT handle multi-segment objects
    multirecording_1 = day_recording # The `concatenate_recordings()` mimics a mono-segment object that concatenates all segments
    multirecording.append(day_recording) # load your recording using SpikeInterface
    multi_shank = gen_probe()
    multirecording_filter = spre.bandpass_filter(multirecording[i], freq_min=300, freq_max=6000)
    multirecording_f.append(multirecording_filter)

In [20]:

# # Define the thresholds for each recording and each channel
# # thresholds = [
# #     [0.0015, 0.003, 0.0045],  # Thresholds for recording 1
# #     [0.002, 0.002, 0.0055],  # Thresholds for recording 2
# #     [0.0055, 0.0025, 0.005],  # Thresholds for recording 3
# #     [0.006, 0.006, 0.003],  # Thresholds for recording 4
# #     [0.006, 0.006, 0.003],  # Thresholds for recording 5
# #     [0.006, 0.006, 0.003],  # Thresholds for recording 6
# #     [0.006, 0.006, 0.003]   # Thresholds for recording 7
# # ]
# thresholds = [
#     [0.0003, 0.0004, 0.0004],  # Thresholds for recording 1
#     [0.0003, 0.0004, 0.0004],  # Thresholds for recording 2
#     [0.0005, 0.0004, 0.0003],  # Thresholds for recording 3
#     [0.0005, 0.0005, 0.0003],  # Thresholds for recording 4
#     [0.0005, 0.0005, 0.0003],  # Thresholds for recording 5
#     [0.0005, 0.0005, 0.0003],  # Thresholds for recording 6
#     [0.0005, 0.0005, 0.0003],  # Thresholds for recording 7
# ]

# ms_before = [100]
# ms_after = [100]
# # Define the range for plotting
# # start_index = int(11.6751 * 1e6)
# # end_index = int(11.7747 * 1e6)
# start_index = int(9.897 * 1e6)
# end_index = int(10 * 1e6)
# artifact_frames = []
# for i in range(len(recording_names)):
#     artifact_frames_per_recording = []  # Store artifact frames for each channel in a recording
    
#     for ch in range(3):  # Assuming there are 3 channels
#         channel = multirecording[i].get_traces()[:, ch]
        
#         # Plot the data in the specified range
#         plt.figure()
#         plt.plot(channel[start_index:end_index])  # Plotting the data between 2.5x10^6 and 3x10^6
        
#         # Use predefined threshold for the current recording and channel
#         threshold = thresholds[i][ch]
#         print(f"Threshold for recording {i+1}, channel {ch+1}: {threshold}")
        
#         # Apply the custom threshold to the entire channel data
#         stimulation_trigger_frames = np.where(np.abs(channel) > threshold)
        
#         # Remove artifacts using the detected triggers
#         multirecording_art = spre.remove_artifacts(
#             multirecording[i],
#             list_triggers=stimulation_trigger_frames[0].tolist(),
#             ms_before=ms_before[0],  
#             ms_after=ms_after[0]
#         )
        
#         # Plot the artifact-removed data in the specified range
#         plt.plot(multirecording_art.get_traces()[:, ch][start_index:end_index])  # Ensure the recording is preprocessed appropriately
#         plt.show()
    
#     artifact_frames.append(artifact_frames_per_recording)


In [ ]:
ms_before = [100]
ms_after = [100]
#Define the thresholds for each recording and each channel
thresholds = [
    [0.00025, 0.0004, 0.0004],  # Thresholds for recording 1
    [0.0002, 0.00035, 0.0004],  # Thresholds for recording 2
    [0.00045, 0.00035, 0.0003],  # Thresholds for recording 3
    [0.00045, 0.0005, 0.00025],  # Thresholds for recording 4
    [0.00045, 0.0005, 0.00025],  # Thresholds for recording 5
    [0.00045, 0.0005, 0.00027],  # Thresholds for recording 6
    [0.00045, 0.0005, 0.0003]  # Thresholds for recording 7
]


artifact_frames = []
for i in range(len(recording_names)):
    artifact_frames_per_recording = []  # Store artifact frames for each channel in a recording
    
    for ch in range(3):  # Assuming there are 3 channels
        channel = multirecording[i].get_traces()[:, ch].astype(np.float32)
        #plt.figure()
        #plt.plot(channel)
        
        # Use predefined threshold for the current recording and channel
        threshold = thresholds[i][ch]
        print(f"Threshold for recording {i+1}, channel {ch+1}: {threshold}")
        
        # Apply the custom threshold
        stimulation_trigger_frames = np.where(np.abs(channel) > threshold)
        
        # Remove artifacts using the detected triggers
        multirecording_art = spre.remove_artifacts(
            multirecording[i],
            list_triggers=stimulation_trigger_frames[0].tolist(),
            ms_before=ms_before[0],  
            ms_after=ms_after[0]
        )
        
        artifact_frames_per_recording.append(stimulation_trigger_frames[0].tolist())
        #plt.plot(multirecording_art.get_traces()[:, ch])  # Ensure the recording is preprocessed appropriately
        #plt.show()
    
    artifact_frames.append(artifact_frames_per_recording)




In [22]:
# from spikeinterface.extractors import NumpyRecording


# # Lazy preprocessing
# recording_art = []
# recording_lfp = []
# recording_mua = []
# recording_delta = []
# recording_100_200 = []
# recording_200_400 = []
# recording_gamma = []
# artifact_frames_resample = []

# # Define the notch filter parameters
# fs = 1000  # Resampled data to 1000 Hz
# f0_60 =60
# f0_180 = 180.0  # Power line noise at 60 Hz
# Q = 50  # Quality factor for the notch filter
# Q1=300
# Q2 = 1000
# f_300 =300
# #Create a notch filter for 60 Hz
# b_60, a_60 = signal.iirnotch(f0_60, 20, 30000)
# b_200, a_200 = signal.iirnotch(f0_180, Q, 30000)
# b_300, a_300 = signal.iirnotch(f_300, Q1, 30000)
# for i in range(len(recording_names)):
#     rc=[]
#     for ch in range(3):
    
#         recording_artifact_removed = spre.remove_artifacts(
#            multirecording[i],
#            list_triggers=artifact_frames[i][ch],  # Combine triggers from all channels
#            ms_before=ms_before[0], ms_after=ms_after[0]  # Adjust this based on your needs  
#         )
#         channel_data_artifact_removed = recording_artifact_removed.get_traces()[:, ch]

#         rc.append(channel_data_artifact_removed)
    
#     # Combine the channels into one array
#     combined_data = np.stack(rc, axis=1)  # Stack along the channel axis

#     # Create a new RecordingExtractor object with combined data
#     sampling_frequency = multirecording[i].get_sampling_frequency()
#     combined_recording = NumpyRecording(traces_list=[combined_data], sampling_frequency=sampling_frequency)
    
#     # Append the filtered recording to the list
#     recording_art.append(combined_recording)
#     # Apply the 60Hz, 200Hz, and 300Hz notch filters to remove noise
#     traces_filtered = signal.filtfilt(b_60, a_60, recording_art[i].get_traces(), axis=0)
#     traces_filtered = signal.filtfilt(b_60, a_60, traces_filtered, axis=0)
#     traces_filtered = signal.filtfilt(b_60, a_60, traces_filtered, axis=0)
#     traces_filtered = signal.filtfilt(b_200, a_200, traces_filtered, axis=0)
#     traces_filtered = signal.filtfilt(b_300, a_300, traces_filtered, axis=0)

#     # Use NumpyRecording to create a new RecordingExtractor with the filtered traces
#     sampling_frequency = recording_artifact_removed.get_sampling_frequency()
#     channel_ids = recording_artifact_removed.get_channel_ids()
#     recording_filtered = se.NumpyRecording([traces_filtered], sampling_frequency, channel_ids=channel_ids)

#     # Process LFP and MUA after filtering
#     lfp, mua = downsample_lfp_mua(recording_filtered)

#     # total_samples = lfp.get_num_samples()

#     # # Apply the 60Hz notch filter multiple times to effectively remove the noise
#     # traces_filtered = lfp.get_traces().astype(np.float32)

#     # traces_filtered = signal.filtfilt(b_60, a_60, traces_filtered, axis=0)

#     # # Use NumpyRecording to create a new RecordingExtractor with the filtered traces
#     # sampling_frequency = lfp.get_sampling_frequency()
#     # channel_ids = lfp.get_channel_ids()
#     # lfp = se.NumpyRecording([traces_filtered], sampling_frequency, channel_ids=channel_ids)

#     delta, s100_200, s200_400, gamma = filterbank(lfp)
#     recording_lfp.append(lfp)
#     recording_mua.append(mua)
#     recording_delta.append(delta)
#     recording_100_200.append(s100_200)
#     recording_200_400.append(s200_400)
#     recording_gamma.append(gamma)
    
#     # Resample artifact frames for further analysis
#     stim_frames = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames[i])))
#     artifact_frames_resample.append(stim_frames)

# time_pre_samples = num_pre_samples / 30000



In [23]:
from scipy import signal
from spikeinterface.extractors import NumpyRecording
import numpy as np

recording_art = []
recording_lfp = []
recording_mua = []
recording_delta = []
recording_100_200 = []
recording_200_400 = []
recording_gamma = []
artifact_frames_resample = []

# Create notch filters for specific frequencies
#frequencies = [1.4, 9.0, 18.2, 25.0, 27.2, 29.2, 36.4, 37.6, 41.4, 43.8, 45.4, 46.4, 60.0, 180.0, 300.0]
frequencies = [60.0, 180.0, 300.0]
Q_values = [60.0, 180.0, 300.0]  # Define Q values corresponding to each frequency

notch_filters = [signal.iirnotch(f, Q, 30000) for f, Q in zip(frequencies, Q_values)]


for i in range(len(recording_names)):
    rc = []
    for ch in range(3):
        # Remove artifacts for each channel
        recording_artifact_removed = spre.remove_artifacts(
            multirecording[i],
            list_triggers=artifact_frames[i][ch],  # Combine triggers from all channels
            ms_before=ms_before[0], ms_after=ms_after[0]  # Adjust based on your needs
        )
        channel_data_artifact_removed = recording_artifact_removed.get_traces()[:, ch]

        # Append the artifact-removed channel data to rc list
        rc.append(channel_data_artifact_removed)

    # Combine the channels into one array
    combined_data = np.stack(rc, axis=1)  # Stack along the channel axis

    # Create a new RecordingExtractor object with combined data
    sampling_frequency = multirecording[i].get_sampling_frequency()
    combined_recording = NumpyRecording(traces_list=[combined_data], sampling_frequency=sampling_frequency)

    # Append the filtered recording to the list
    recording_art.append(combined_recording)

    # Apply notch filters sequentially to remove noise at specified frequencies
    traces_filtered = recording_art[i].get_traces()
    for b, a in notch_filters:
        traces_filtered = signal.filtfilt(b, a, traces_filtered, axis=0)

    # Create a new RecordingExtractor with the filtered traces
    recording_filtered = NumpyRecording([traces_filtered], sampling_frequency)

    # Process LFP and MUA after filtering
    lfp, mua = downsample_lfp_mua(recording_filtered)

    # Apply filter bank to LFP
    delta, s100_200, s200_400, gamma = filterbank(lfp)
    recording_lfp.append(lfp)
    recording_mua.append(mua)
    recording_delta.append(delta)
    recording_100_200.append(s100_200)
    recording_200_400.append(s200_400)
    recording_gamma.append(gamma)

    # Resample artifact frames for further analysis
    stim_frames = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames[i])))
    artifact_frames_resample.append(stim_frames)

In [ ]:
import numpy as np
import scipy.signal as sp_signal  # Rename scipy.signal to avoid conflicts
from spikeinterface.extractors import NumpyRecording
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import hilbert 

# Helper functions
def extract_phase(signal):
    """Extract the instantaneous phase of a signal using Hilbert transform"""
    analytic_signal = hilbert(signal)
    phase = np.angle(analytic_signal)
    return phase

def calculate_plv(phases):
    """Compute Phase Locking Value (PLV) between multiple signals"""
    n_signals = phases.shape[0]
    plv_matrix = np.zeros((n_signals, n_signals))
    for i in range(n_signals):
        for j in range(i + 1, n_signals):
            phase_diff = phases[i] - phases[j]
            plv_matrix[i, j] = np.abs(np.mean(np.exp(1j * phase_diff)))
            plv_matrix[j, i] = plv_matrix[i, j]
    return plv_matrix

def bandpass_filter(signal, lowcut, highcut, fs, order=4):
    """Apply a bandpass filter"""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = sp_signal.butter(order, [low, high], btype="band")
    filtered_signal = sp_signal.filtfilt(b, a, signal, axis=0)
    return filtered_signal

def calculate_band_plv(traces_filtered, fs, lowcut, highcut):
    """Compute PLV for a specific frequency band"""
    band_filtered = np.array([bandpass_filter(traces_filtered[:, ch], lowcut, highcut, fs) for ch in range(traces_filtered.shape[1])])
    phases = np.array([extract_phase(band_filtered[ch]) for ch in range(band_filtered.shape[0])])
    plv_matrix = calculate_plv(phases)
    return plv_matrix


# Iterate over recordings
for i in range(len(recording_art)):
    # Extract LFP signal
    lfp = recording_lfp[i].get_traces()
    fs = recording_lfp[i].get_sampling_frequency()
    start_time = stim_start_array[i][1] + 0.7  # Start time (seconds)
    end_time = start_time + 1.5  # End time (seconds)

    start_sample = int(start_time * fs)
    end_sample = int(end_time * fs)
    lfp = lfp[start_sample:end_sample, :] 
    fs = recording_lfp[i].get_sampling_frequency()

    # Compute original PLV for the 1-10 Hz band
    plv_5_20hz_original = calculate_band_plv(lfp, fs, 1, 10)
    print(f"Recording {i}: Original PLV for 5-20 Hz:\n{plv_5_20hz_original}")

    lfp_notch_filtered = lfp.copy()

    # Compute PLV after Notch filtering for 11-20 Hz
    plv_5_20hz_filtered = calculate_band_plv(lfp_notch_filtered, fs, 11, 20.0)
    print(f"Recording {i}: PLV for 5-20 Hz after Notch Filtering:\n{plv_5_20hz_filtered}")

    # Function to visualize the PLV matrix
    def visualize_plv(plv_matrix, title):
        plt.figure(figsize=(6, 4))
        sns.heatmap(plv_matrix, annot=True, cmap="viridis", xticklabels=["Ch1", "Ch2", "Ch3"], yticklabels=["Ch1", "Ch2", "Ch3"])
        plt.title(title)
        plt.xlabel("Channels")
        plt.ylabel("Channels")
        plt.show()

    # Visualize PLV matrices for original and filtered signals
    visualize_plv(plv_5_20hz_original, f"Original PLV for 5-20 Hz (Recording {i})")
    visualize_plv(plv_5_20hz_filtered, f"PLV for 5-20 Hz after Notch Filtering (Recording {i})")



In [ ]:
import numpy as np
import scipy.signal as sp_signal  # Rename scipy.signal to avoid conflicts
from spikeinterface.extractors import NumpyRecording
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import hilbert 

# Helper functions
def extract_phase(signal):
    """Extract the instantaneous phase of a signal using the Hilbert transform"""
    analytic_signal = hilbert(signal)
    phase = np.angle(analytic_signal)
    return phase

def calculate_pli(phases):
    """Compute Phase Lag Index (PLI) between multiple signals"""
    n_signals = phases.shape[0]
    pli_matrix = np.zeros((n_signals, n_signals))
    for i in range(n_signals):
        for j in range(i + 1, n_signals):
            phase_diff = phases[i] - phases[j]
            pli_matrix[i, j] = np.abs(np.mean(np.sign(np.sin(phase_diff))))
            pli_matrix[j, i] = pli_matrix[i, j]
    return pli_matrix

def bandpass_filter(signal, lowcut, highcut, fs, order=4):
    """Apply a bandpass filter"""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = sp_signal.butter(order, [low, high], btype="band")
    filtered_signal = sp_signal.filtfilt(b, a, signal, axis=0)
    return filtered_signal

def calculate_band_pli(traces_filtered, fs, lowcut, highcut):
    """Compute PLI for a specific frequency band"""
    band_filtered = np.array([bandpass_filter(traces_filtered[:, ch], lowcut, highcut, fs) for ch in range(traces_filtered.shape[1])])
    phases = np.array([extract_phase(band_filtered[ch]) for ch in range(band_filtered.shape[0])])
    pli_matrix = calculate_pli(phases)
    return pli_matrix

# Iterate over recordings
for i in range(len(recording_art)):
    # Extract LFP signal
    lfp = recording_lfp[i].get_traces()
    fs = recording_lfp[i].get_sampling_frequency()
    start_time = stim_start_array[i][1] + 0.7  # Start time (seconds)
    end_time = start_time + 1.5  # End time (seconds)

    start_sample = int(start_time * fs)
    end_sample = int(end_time * fs)
    lfp = lfp[start_sample:end_sample, :] 

    # Step 1: Compute PLI for the 1-10 Hz original signal
    pli_1_10hz_original = calculate_band_pli(lfp, fs, 55, 65)
    print(f"Recording {i}: Original PLI for 1-10 Hz:\n{pli_1_10hz_original}")

    # Step 2: Compute PLI for the 11-20 Hz filtered signal
    pli_11_20hz_filtered = calculate_band_pli(lfp, fs, 50, 60)
    print(f"Recording {i}: PLI for 11-20 Hz:\n{pli_11_20hz_filtered}")

    # Function to visualize the PLI matrix
    def visualize_pli(pli_matrix, title):
        plt.figure(figsize=(6, 4))
        sns.heatmap(pli_matrix, annot=True, cmap="coolwarm", xticklabels=["Ch1", "Ch2", "Ch3"], yticklabels=["Ch1", "Ch2", "Ch3"])
        plt.title(title)
        plt.xlabel("Channels")
        plt.ylabel("Channels")
        plt.show()

    # Visualize PLI matrices for the original and filtered signals
    visualize_pli(pli_1_10hz_original, f"Original PLI for 1-10 Hz (Recording {i})")
    visualize_pli(pli_11_20hz_filtered, f"PLI for 11-20 Hz (Recording {i})")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Extract LFP data from the preprocessed recording_lfp
lfp_data = recording_lfp[1].get_traces()  # Assuming recording_lfp[0] is a RecordingExtractor

# Check the shape of lfp_data to confirm its dimensions
print(f"LFP data shape: {lfp_data.shape}")  # Output the shape to confirm it is 2D (num_samples, num_channels)

# If the LFP data contains multiple channels, select one channel for analysis, e.g., the first channel
lfp_channel_data = lfp_data[:, 0]  # Select the first channel

# Step 1: Compute the power spectral density (PSD) for the 1-20 Hz band
f, Pxx = signal.welch(lfp_channel_data, fs=1000, nperseg=400)

# Step 2: Plot the power spectral density to identify noise in the 1-20 Hz band
plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx)
plt.xlim([1, 400])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density [V^2/Hz]')
plt.title('Power Spectral Density (1-20 Hz)')
plt.grid(True)
plt.show()

# Step 3: Use STFT to analyze frequency components in the 1-20 Hz range
f_stft, t_stft, Zxx = signal.stft(lfp_channel_data, fs=1000, nperseg=400)

# Plot the STFT results
plt.figure(figsize=(10, 6))
plt.pcolormesh(t_stft, f_stft, np.abs(Zxx), shading='gouraud')
plt.colorbar(label='Magnitude')
plt.ylim([1, 50])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Time [sec]')
plt.ylabel('Frequency [Hz]')
plt.title('STFT Magnitude (1-20 Hz)')
plt.show()


In [13]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy import signal

# # Extract LFP data from the preprocessed recording_lfp
# lfp_data = recording_lfp[0].get_traces()  # Assuming recording_lfp[0] is a RecordingExtractor object

# # Check the shape of lfp_data to confirm its dimensions
# print(f"LFP data shape: {lfp_data.shape}")  # Output the shape to confirm it is 2D (num_samples, num_channels)

# # If the LFP data contains multiple channels, select one channel for analysis, e.g., the first channel
# lfp_channel_data = lfp_data[:, 0]  # Select the first channel

# fs = 1000  # Assume the sampling rate is 1000 Hz

# # Define notch filter frequencies
# f0_60 = 60  # 60 Hz noise frequency
# f0_200 = 180  # First noise frequency at 200 Hz
# f0_300 = 300.0  # Second noise frequency at 300 Hz
# Q = 100  # Quality factor of the notch filter, controlling bandwidth

# # Create notch filters for 60 Hz, 200 Hz, and 300 Hz
# b_60, a_60 = signal.iirnotch(f0_60, Q, fs)
# b_200, a_200 = signal.iirnotch(f0_200, Q, fs)
# b_300, a_300 = signal.iirnotch(f0_300, Q, fs)

# # Apply notch filters to remove noise at 60 Hz, 200 Hz, and 300 Hz
# filtered_lfp_data = signal.filtfilt(b_60, a_60, lfp_channel_data)
# filtered_lfp_data = signal.filtfilt(b_200, a_200, filtered_lfp_data)
# filtered_lfp_data = signal.filtfilt(b_300, a_300, filtered_lfp_data)

# # Step 1: Compute the power spectral density (PSD) of the filtered signal
# f_filtered, Pxx_filtered = signal.welch(filtered_lfp_data, fs=fs, nperseg=400)

# # Step 2: Plot the power spectral density after filtering
# plt.figure(figsize=(10, 6))
# plt.semilogy(f_filtered, Pxx_filtered)
# plt.xlim([1, 400])  # Limit the frequency range to 5-400 Hz
# plt.xlabel('Frequency [Hz]')
# plt.ylabel('Power Spectral Density [V^2/Hz]')
# plt.title('Filtered Power Spectral Density (5-400 Hz)')
# plt.grid(True)
# plt.show()

# # Step 3: Use STFT to analyze frequency components in the 5-400 Hz range
# f_stft, t_stft, Zxx = signal.stft(filtered_lfp_data, fs=fs, nperseg=400)

# # Plot the STFT results
# plt.figure(figsize=(10, 6))
# plt.pcolormesh(t_stft, f_stft, np.abs(Zxx), shading='gouraud')
# plt.colorbar(label='Magnitude')
# plt.ylim([1, 400])  # Limit the frequency range to 5-400 Hz
# plt.xlabel('Time [sec]')
# plt.ylabel('Frequency [Hz]')
# plt.title('Filtered STFT Magnitude (5-400 Hz)')
# plt.show()



5uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[0].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[0].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[500000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 29  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[0].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[6000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[0].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

10uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[1].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[1].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[1].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[1].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

20uA


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[2].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1200000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[2].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], :])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[2].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[2].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

30uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60  + 25# 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[3].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[3].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[3].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 59  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[3].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[7000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

40uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[4].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[4].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[4].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

50uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[5].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[5].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[1000000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5* 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[5].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

60uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data = multirecording[6].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[1500000], time_axis[2300000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 30000  # Sampling frequency (30kHz)
time_window = 5 * 60 +25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_art[6].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[1500000], time_axis[2500000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[6].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 5 * 60 + 25 # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[6].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
#    axs[i].set_xlim([time_axis[0], time_axis[35109]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

5_c


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data
nperseg = 0.25*fs_stim  # Window size for coherence calculation
time_buffer = 0.6
 #seconds before/after stimulus, to avoid artifacts
overlap = int(0.5 * nperseg)  # 25% overlap of the window size
time_period =  1.5 #seconds
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[0][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[0].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg1 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg1[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()


10_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[1][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[1].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg2 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg2[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

20_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[2][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[2].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg3 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg3[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

30_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[3][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[3].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg4 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg4[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()


40_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[4][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[4].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg5 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg5[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

50_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[5][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[5].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg6 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg6[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

60_c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[6][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[6].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f, Cxy_post = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum[:, j] += Cxy_post  # Accumulate coherence values
        
        axs[i, j].semilogy(f, Cxy_post)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg7 = coherence_sum / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f, coherence_avg7[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize an array to store the average coherence values for all recordings
all_coherence_avg = np.zeros((int(nperseg / 2 + 1), 7, 3))  # 7 recordings, 3 coherence pairs

# Assign the precomputed coherence averages to the all_coherence_avg array
all_coherence_avg[:, 0, :] = coherence_avg1
all_coherence_avg[:, 1, :] = coherence_avg2
all_coherence_avg[:, 2, :] = coherence_avg3
all_coherence_avg[:, 3, :] = coherence_avg4
all_coherence_avg[:, 4, :] = coherence_avg5
all_coherence_avg[:, 5, :] = coherence_avg6
all_coherence_avg[:, 6, :] = coherence_avg7

# Plot average coherence curves in separate figures for each pair
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
labels = [f'{i}' for i in amps]

for j in range(3):  # Loop over each of the three coherence pairs
    fig, ax = plt.subplots(dpi=400, figsize=(5, 5))

    for k in range(7):
        ax.semilogy(f, all_coherence_avg[:, k, j], label=labels[k], color=colors[k])

    ax.set_title(f'Average Coherence - {titles[j]}')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Coherence')
    ax.set_xlim([1, 50])
    ax.set_ylim([10**(-3), 10**(0)])
    ax.legend(loc='lower right')

    plt.tight_layout()
    plt.show()


In [ ]:
stim_times_array1 = []
for i in range(len(recording_names1)):
    BO7_res1 = pull_files(intan_folders1[i], intan_path1)
    stim_names_BO71 = ['STIM_A-000','STIM_A-002','STIM_A-004']
    stim_BO71 = all_intan_data(stim_names_BO71,'stim_data',BO7_res1)
    stim_times1 = stim_BO71.loc[(stim_BO71['STIM_A-000'] != 0) | (stim_BO71['STIM_A-002'] != 0) | (stim_BO71['STIM_A-004'] != 0)].index
    stim_times_array1.append(stim_times1.to_numpy())

In [ ]:
stim_start_array1 = []
stim_end_array1 = []
for stim in stim_times_array1:
    stim = np.insert(stim,0,0)
    stim_times_diff1 = np.diff(stim)
    stim_start1 = stim[np.where(stim_times_diff1 > 1)[0]+1]
    stim_end1 = stim[np.where(stim_times_diff1 > 1)[0][1:]]
    stim_end1 = np.concatenate((stim_end1,[stim[-1]]))
    print(stim_start1)
    print(stim_end1)
    stim_start_array1.append(stim_start1)
    stim_end_array1.append(stim_end1)

In [ ]:

num_samples1 = []
num_channels1 = []
multirecording1 = []
multirecording_f1 = []
for i in range(len(recording_names1)):
    recording_names1 = [folder + 'BO11_O7W_Sti_Day4_' + amp.split('u')[0] + 'uA_m.nwb' 
                       for folder, amp in zip(intan_folders1, amps1)]
    rec_names1 = intan_path1 + recording_names1[i]
    day_recording1 = load_data(rec_names1)
    [num_pre_samples1, num_pre_channels1] = day_recording1.get_traces().shape # Num samples, num channels
    num_samples1.append(num_pre_samples1)
    num_channels1.append(num_pre_channels1)
    print('Presample:' + str(num_pre_samples1))

    # Case 2: the sorter DOES NOT handle multi-segment objects
    multirecording_1 = day_recording1 # The `concatenate_recordings()` mimics a mono-segment object that concatenates all segments
    multirecording1.append(day_recording1) # load your recording using SpikeInterface
    multi_shank1 = gen_probe()
    multirecording_filter1 = spre.bandpass_filter(multirecording1[i], freq_min=300, freq_max=6000)
    multirecording_f1.append(multirecording_filter1)

In [ ]:
ms_before = [100]
ms_after = [100]

#Define the thresholds for each recording and each channel
thresholds1 = [
    [0.00005, 0.0002, 0.00035],  # Thresholds for recording 1
    [0.00025, 0.0004, 0.00037],  # Thresholds for recording 2
    [0.0003, 0.0004, 0.0005],  # Thresholds for recording 3
    [0.0003, 0.0005, 0.0005],  # Thresholds for recording 4
    [0.0003, 0.0005, 0.0005],  # Thresholds for recording 5
    [0.0004, 0.0005, 0.0005],  # Thresholds for recording 6
    [0.0004, 0.0005, 0.0005]  # Thresholds for recording 7
]

artifact_frames1 = []
for i in range(len(recording_names1)):
    artifact_frames_per_recording1 = []  # Store artifact frames for each channel in a recording
    
    for ch in range(3):  # Assuming there are 3 channels
        channel1 = multirecording1[i].get_traces()[:, ch].astype(np.float32)
        #plt.figure()
        #plt.plot(channel)
        
        # Use predefined threshold for the current recording and channel
        threshold1 = thresholds1[i][ch]
        print(f"Threshold for recording {i+1}, channel {ch+1}: {threshold1}")
        
        # Apply the custom threshold
        stimulation_trigger_frames1 = np.where(np.abs(channel1) > threshold1)
        
        # Remove artifacts using the detected triggers
        multirecording_art1 = spre.remove_artifacts(
            multirecording1[i],
            list_triggers=stimulation_trigger_frames1[0].tolist(),
            ms_before=ms_before[0],  
            ms_after=ms_after[0]
        )
        
        artifact_frames_per_recording1.append(stimulation_trigger_frames1[0].tolist())
        #plt.plot(multirecording_art.get_traces()[:, ch])  # Ensure the recording is preprocessed appropriately
        #plt.show()
    
    artifact_frames1.append(artifact_frames_per_recording1)


In [54]:
from scipy import signal
from spikeinterface.extractors import NumpyRecording
import numpy as np

recording_art1 = []
recording_lfp1 = []
recording_mua1= []
recording_delta1 = []
recording_100_2001 = []
recording_200_4001 = []
recording_gamma1 = []
artifact_frames_resample1 = []

# Create notch filters for specific frequencies
#frequencies = [1.4, 9.0, 18.2, 25.0, 27.2, 29.2, 36.4, 37.6, 41.4, 43.8, 45.4, 46.4, 60.0, 180.0, 300.0]
frequencies = [60.0, 180.0, 300.0]
Q_values = [60.0, 180.0, 300.0]  # Define Q values corresponding to each frequency

notch_filters = [signal.iirnotch(f, Q, 30000) for f, Q in zip(frequencies, Q_values)]


for i in range(len(recording_names1)):
    rc1 = []
    for ch in range(3):
        # Remove artifacts for each channel
        recording_artifact_removed1 = spre.remove_artifacts(
            multirecording1[i],
            list_triggers=artifact_frames1[i][ch],  # Combine triggers from all channels
            ms_before=ms_before[0], ms_after=ms_after[0]  # Adjust based on your needs
        )
        channel_data_artifact_removed1 = recording_artifact_removed1.get_traces()[:, ch]

        # Append the artifact-removed channel data to rc list
        rc1.append(channel_data_artifact_removed1)

    # Combine the channels into one array
    combined_data1 = np.stack(rc1, axis=1)  # Stack along the channel axis

    # Create a new RecordingExtractor object with combined data
    sampling_frequency = multirecording1[i].get_sampling_frequency()
    combined_recording1 = NumpyRecording(traces_list=[combined_data1], sampling_frequency=sampling_frequency)

    # Append the filtered recording to the list
    recording_art1.append(combined_recording1)

    # Apply notch filters sequentially to remove noise at specified frequencies
    traces_filtered = recording_art1[i].get_traces()
    for b, a in notch_filters:
        traces_filtered = signal.filtfilt(b, a, traces_filtered, axis=0)

    # Create a new RecordingExtractor with the filtered traces
    recording_filtered1 = NumpyRecording([traces_filtered], sampling_frequency)

    # Process LFP and MUA after filtering
    lfp1, mua1 = downsample_lfp_mua(recording_filtered1)

    # Apply filter bank to LFP
    delta1, s100_2001, s200_4001, gamma1 = filterbank(lfp)
    recording_lfp1.append(lfp1)
    recording_mua1.append(mua1)
    recording_delta1.append(delta1)
    recording_100_2001.append(s100_2001)
    recording_200_4001.append(s200_4001)
    recording_gamma1.append(gamma1)

    # Resample artifact frames for further analysis
    stim_frames1 = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames1[i])))
    artifact_frames_resample1.append(stim_frames1)


In [55]:
# from spikeinterface.extractors import NumpyRecording
# import numpy as np
# import pywt

# # Define a wavelet denoising function
# def wavelet_denoise(data, wavelet='db4', level=4):
#     # Perform wavelet decomposition
#     coeffs = pywt.wavedec(data, wavelet, mode='per', level=level)
#     # Set denoising threshold
#     sigma = (1/0.6745) * np.median(np.abs(coeffs[-1] - np.median(coeffs[-1])))
#     uthresh = sigma * np.sqrt(2 * np.log(len(data)))
#     # Apply soft thresholding
#     coeffs[1:] = [pywt.threshold(i, value=uthresh, mode='soft') for i in coeffs[1:]]
#     # Reconstruct the signal
#     reconstructed_signal = pywt.waverec(coeffs, wavelet, mode='per')
#     return reconstructed_signal

# # Data preprocessing and analysis
# recording_art1 = []
# recording_lfp1 = []
# recording_mua1 = []
# recording_delta1 = []
# recording_100_2001 = []
# recording_200_4001 = []
# recording_gamma1 = []
# artifact_frames_resample1 = []

# for i in range(len(recording_names1)):
#     rc1 = []
#     for ch in range(3):
#         # Remove artifacts from the recording
#         recording_artifact_removed1 = spre.remove_artifacts(
#             multirecording1[i],
#             list_triggers=artifact_frames1[i][ch],
#             ms_before=ms_before[0], ms_after=ms_after[0]
#         )
#         channel_data_artifact_removed1 = recording_artifact_removed1.get_traces()[:, ch]

#         # Apply wavelet denoising
#         channel_data_artifact_removed1 = wavelet_denoise(channel_data_artifact_removed1)

#         # Add single-channel data to the rc list
#         rc1.append(channel_data_artifact_removed1)

#     # Combine the channels into one array
#     combined_data1 = np.stack(rc1, axis=1)

#     # Create a new RecordingExtractor object
#     combined_recording1 = NumpyRecording(traces_list=[combined_data1], sampling_frequency=multirecording1[i].get_sampling_frequency())
#     recording_art1.append(combined_recording1)

#     # Process LFP and MUA after wavelet denoising
#     lfp1, mua1 = downsample_lfp_mua(combined_recording1)

#     # Apply filter bank processing to extract frequency bands
#     delta1, s100_2001, s200_4001, gamma1 = filterbank(lfp1)
#     recording_lfp1.append(lfp1)
#     recording_mua1.append(mua1)
#     recording_delta1.append(delta1)
#     recording_100_2001.append(s100_2001)
#     recording_200_4001.append(s200_4001)
#     recording_gamma1.append(gamma1)

#     # Resample artifact frames for further analysis
#     stim_frames1 = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames1[i])))
#     artifact_frames_resample1.append(stim_frames1)

# # Output: Data processing complete


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Extract LFP data from the preprocessed recording_lfp
lfp_data1 = recording_lfp1[0].get_traces()  # Assuming recording_lfp[0] is a RecordingExtractor

# Check the shape of lfp_data to confirm its dimensions
print(f"LFP data shape: {lfp_data1.shape}")  # Output the shape to confirm it is 2D (num_samples, num_channels)

# If the LFP data contains multiple channels, select one channel for analysis, e.g., the first channel
lfp_channel_data1 = lfp_data1[:, 0]  # Select the first channel

# Step 1: Compute the power spectral density (PSD) for the 1-20 Hz band
f_20, Pxx = signal.welch(lfp_channel_data1, fs=1000, nperseg=400)

# Step 2: Plot the power spectral density to detect noise in the 1-20 Hz band
plt.figure(figsize=(10, 6))
plt.semilogy(f_20, Pxx)
plt.xlim([1, 400])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density [V^2/Hz]')
plt.title('Power Spectral Density (1-20 Hz)')
plt.grid(True)
plt.show()

# Step 3: Use STFT to analyze frequency components in the 1-20 Hz range
f_stft, t_stft, Zxx = signal.stft(lfp_channel_data1, fs=1000, nperseg=400)

# Plot the STFT results
plt.figure(figsize=(10, 6))
plt.pcolormesh(t_stft, f_stft, np.abs(Zxx), shading='gouraud')
plt.colorbar(label='Magnitude')
plt.ylim([1, 400])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Time [sec]')
plt.ylabel('Frequency [Hz]')
plt.title('STFT Magnitude (1-20 Hz)')
plt.show()


5uA


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data
nperseg1 = 0.25*fs_stim  # Window size for coherence calculation
time_buffer1 = 0.6 #seconds before/after stimulus, to avoid artifacts
time_period = 1.5 #seconds
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[0][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[0].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg11 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg11[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()


10uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[1][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[1].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg12 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg12[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()


20uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[2][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[2].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg13 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg13[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

30uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[3][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[3].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg14 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg14[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

40uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[4][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[4].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg15 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg15[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

50uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[5][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[5].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg16 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg16[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

60uA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns

coherence_sum1 = np.zeros((int(nperseg1/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array1[6][:5]):  # Iterate over the first 5 pulse times
    segment1 = recording_lfp1[6].get_traces()[int(pulse_time * fs_stim + time_buffer1 * fs_stim):int((pulse_time + time_buffer1 + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post1 = segment1[:, ind1[j]]
        y_post1 = segment1[:, ind2[j]]
        f1, Cxy_post1 = signal.coherence(x_post1, y_post1, fs_stim, nperseg=nperseg1, noverlap=overlap)
        
        coherence_sum1[:, j] += Cxy_post1  # Accumulate coherence values
        
        axs[i, j].semilogy(f1, Cxy_post1)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([1, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg17 = coherence_sum1 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f1, coherence_avg17[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([1, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize an array to store the average coherence values for all recordings
all_coherence_avg1 = np.zeros((int(nperseg1 / 2 + 1), 7, 3))  # 7 recordings, 3 coherence pairs

# Assign the precomputed coherence averages to the all_coherence_avg array
all_coherence_avg1[:, 0, :] = coherence_avg11
all_coherence_avg1[:, 1, :] = coherence_avg12
all_coherence_avg1[:, 2, :] = coherence_avg13
all_coherence_avg1[:, 3, :] = coherence_avg14
all_coherence_avg1[:, 4, :] = coherence_avg15
all_coherence_avg1[:, 5, :] = coherence_avg16
all_coherence_avg1[:, 6, :] = coherence_avg17

# Plot average coherence curves in separate figures for each pair
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
labels = [f'{i}' for i in amps]

for j in range(3):  # Loop over each of the three coherence pairs
    fig, ax = plt.subplots(dpi=400, figsize=(5, 5))

    for k in range(7):
        ax.semilogy(f1, all_coherence_avg1[:, k, j], label=labels[k], color=colors[k])

    ax.set_title(f'Average Coherence - {titles[j]}')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Coherence')
    ax.set_xlim([1, 50])
    ax.set_ylim([10**(-3), 10**(0)])
    ax.legend(loc='lower right')

    plt.tight_layout()
    plt.show()


In [67]:
recording_names = ['BO5_O7W_Sti_Day4_5uA_m.nwb',
                   'BO5_O7W_Sti_Day4_10uA_m.nwb',
                   'BO5_O7W_Sti_Day4_20uA_m.nwb',
                   'BO5_O7W_Sti_Day4_30uA_m.nwb',
                   'BO5_O7W_Sti_Day4_40uA_m.nwb',
                   'BO5_O7W_Sti_Day4_50uA_m.nwb',
                   'BO5_O7W_Sti_Day4_60uA_m.nwb'
                   ]

amps = ['5ua','10ua','20ua',
        '30ua','40ua','50ua','60ua']
intan_path = 'C:/Users/27707/Documents/jhu_master/LFP_analysis/2024-07-26_DG_3elec/Experimental Set_Final Round_Batch 27/7 Week Old Organoid/Stimulation Day1/'
intan_folders = ['5uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_5uA_240708_122554/merged/',
                 '10uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_10uA_240708_123516/merged/',
                 '20uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_20uA_240708_124435/merged/',
                 '30uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_30uA_240708_125400/merged/',
                 '40uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_40uA_240708_130304/merged/',
                 '50uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_50uA_240708_131207/merged/',
                 '60uA_233.3uS_Biphasic Pulse/BO5_O7W_Stimulation_Day4_60uA_240708_132110/merged/'
                 ]

In [ ]:
stim_times_array = []
for i in range(len(recording_names)):
    BO7_res = pull_files(intan_folders[i], intan_path)
    stim_names_BO7 = ['STIM_A-000','STIM_A-002','STIM_A-004']
    stim_BO7 = all_intan_data(stim_names_BO7,'stim_data',BO7_res)
    stim_times = stim_BO7.loc[(stim_BO7['STIM_A-000'] != 0) | (stim_BO7['STIM_A-002'] != 0) | (stim_BO7['STIM_A-004'] != 0)].index
    stim_times_array.append(stim_times.to_numpy())

In [ ]:
stim_start_array = []
stim_end_array = []
for stim in stim_times_array:
    stim = np.insert(stim,0,0)
    stim_times_diff = np.diff(stim)
    stim_start = stim[np.where(stim_times_diff > 1)[0]+1]
    stim_end = stim[np.where(stim_times_diff > 1)[0][1:]]
    stim_end = np.concatenate((stim_end,[stim[-1]]))
    print(stim_start)
    print(stim_end)
    stim_start_array.append(stim_start)
    stim_end_array.append(stim_end)

In [ ]:
num_samples = []
num_channels = []
multirecording = []
multirecording_f = []
for i in range(len(recording_names)):
    recording_names = [folder + 'BO5_O7W_Sti_Day4_' + amp.split('u')[0] + 'uA_240708_m.nwb' 
                       for folder, amp in zip(intan_folders, amps)]
    rec_names = intan_path + recording_names[i]
    day_recording = load_data(rec_names)
    [num_pre_samples, num_pre_channels] = day_recording.get_traces().shape # Num samples, num channels
    num_samples.append(num_pre_samples)
    num_channels.append(num_pre_channels)
    print('Presample:' + str(num_pre_samples))

    # Case 2: the sorter DOES NOT handle multi-segment objects
    multirecording_1 = day_recording # The `concatenate_recordings()` mimics a mono-segment object that concatenates all segments
    multirecording.append(day_recording) # load your recording using SpikeInterface
    multi_shank = gen_probe()
    multirecording_filter = spre.bandpass_filter(multirecording[i], freq_min=300, freq_max=6000)
    multirecording_f.append(multirecording_filter)

In [ ]:
ms_before = [100]
ms_after = [100]
# Define the thresholds for each recording and each channel

thresholds = [
    [0.00005, 0.0002, 0.00035],  # Thresholds for recording 1
    [0.00025, 0.0004, 0.00037],  # Thresholds for recording 2
    [0.00025, 0.00045, 0.0005],  # Thresholds for recording 3
    [0.0003, 0.0005, 0.0005],  # Thresholds for recording 4
    [0.00035, 0.0005, 0.0005],  # Thresholds for recording 5
    [0.0004, 0.0005, 0.0005],  # Thresholds for recording 6
    [0.0004, 0.0005, 0.0005]  # Thresholds for recording 7
]

artifact_frames = []
for i in range(len(recording_names)):
    artifact_frames_per_recording = []  # Store artifact frames for each channel in a recording
    
    for ch in range(3):  # Assuming there are 3 channels
        channel = multirecording[i].get_traces()[:, ch].astype(np.float32)
        # plt.figure()
        # plt.plot(channel)
        
        # Use predefined threshold for the current recording and channel
        threshold = thresholds[i][ch]
        print(f"Threshold for recording {i+1}, channel {ch+1}: {threshold}")
        
        # Apply the custom threshold
        stimulation_trigger_frames = np.where(np.abs(channel) > threshold)
        
        # Remove artifacts using the detected triggers
        multirecording_art = spre.remove_artifacts(
            multirecording[i],
            list_triggers=stimulation_trigger_frames[0].tolist(),
            ms_before=ms_before[0],  
            ms_after=ms_after[0]
        )
        
        artifact_frames_per_recording.append(stimulation_trigger_frames[0].tolist())
        # plt.plot(multirecording_art.get_traces()[:, ch])  # Ensure the recording is preprocessed appropriately
        # plt.show()
    
    artifact_frames.append(artifact_frames_per_recording)


In [72]:
from scipy import signal
from spikeinterface.extractors import NumpyRecording
import numpy as np

recording_art = []
recording_lfp = []
recording_mua = []
recording_delta = []
recording_100_200 = []
recording_200_400 = []
recording_gamma = []
artifact_frames_resample = []

# Create notch filters for specific frequencies
#frequencies = [1.4, 9.0, 18.2, 25.0, 27.2, 29.2, 36.4, 37.6, 41.4, 43.8, 45.4, 46.4, 60.0, 180.0, 300.0]
frequencies = [60.0, 180.0, 300.0]
Q_values = [60.0, 180.0, 300.0]  # Define Q values corresponding to each frequency

notch_filters = [signal.iirnotch(f, Q, 30000) for f, Q in zip(frequencies, Q_values)]


for i in range(len(recording_names)):
    rc = []
    for ch in range(3):
        # Remove artifacts for each channel
        recording_artifact_removed = spre.remove_artifacts(
            multirecording[i],
            list_triggers=artifact_frames[i][ch],  # Combine triggers from all channels
            ms_before=ms_before[0], ms_after=ms_after[0]  # Adjust based on your needs
        )
        channel_data_artifact_removed = recording_artifact_removed.get_traces()[:, ch]

        # Append the artifact-removed channel data to rc list
        rc.append(channel_data_artifact_removed)

    # Combine the channels into one array
    combined_data = np.stack(rc, axis=1)  # Stack along the channel axis

    # Create a new RecordingExtractor object with combined data
    sampling_frequency = multirecording[i].get_sampling_frequency()
    combined_recording = NumpyRecording(traces_list=[combined_data], sampling_frequency=sampling_frequency)

    # Append the filtered recording to the list
    recording_art.append(combined_recording)

    # Apply notch filters sequentially to remove noise at specified frequencies
    traces_filtered = recording_art[i].get_traces()
    for b, a in notch_filters:
        traces_filtered = signal.filtfilt(b, a, traces_filtered, axis=0)

    # Create a new RecordingExtractor with the filtered traces
    recording_filtered = NumpyRecording([traces_filtered], sampling_frequency)

    # Process LFP and MUA after filtering
    lfp, mua = downsample_lfp_mua(recording_filtered)

    # Apply filter bank to LFP
    delta, s100_200, s200_400, gamma = filterbank(lfp)
    recording_lfp.append(lfp)
    recording_mua.append(mua)
    recording_delta.append(delta)
    recording_100_200.append(s100_200)
    recording_200_400.append(s200_400)
    recording_gamma.append(gamma)

    # Resample artifact frames for further analysis
    stim_frames = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames[i])))
    artifact_frames_resample.append(stim_frames)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[0][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[0].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg21 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg21[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[1][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[1].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg22 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg22[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal


fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[2][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[2].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg23 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg23[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[3][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[3].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg24 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg24[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[4][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[4].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg25 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg25[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[5][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[5].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg26 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg26[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum2 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[6][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[6].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f2, Cxy_post2 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum2[:, j] += Cxy_post2  # Accumulate coherence values
        
        axs[i, j].semilogy(f2, Cxy_post2)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg27 = coherence_sum2 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f2, coherence_avg27[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize an array to store the average coherence values for all recordings
all_coherence_avg2 = np.zeros((int(nperseg / 2 + 1), 7, 3))  # 7 recordings, 3 coherence pairs

# Assign the precomputed coherence averages to the all_coherence_avg array
all_coherence_avg2[:, 0, :] = coherence_avg21
all_coherence_avg2[:, 1, :] = coherence_avg22
all_coherence_avg2[:, 2, :] = coherence_avg23
all_coherence_avg2[:, 3, :] = coherence_avg24
all_coherence_avg2[:, 4, :] = coherence_avg25
all_coherence_avg2[:, 5, :] = coherence_avg26
all_coherence_avg2[:, 6, :] = coherence_avg27

# Plot average coherence curves in separate figures for each pair
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
labels = [f'{i}' for i in amps]

for j in range(3):  # Loop over each of the three coherence pairs
    fig, ax = plt.subplots(dpi=400, figsize=(10, 8))

    for k in range(7):
        ax.semilogy(f2, all_coherence_avg2[:, k, j], label=labels[k], color=colors[k])

    ax.set_title(f'Average Coherence - {titles[j]}')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Coherence')
    ax.set_xlim([1, 400])
    ax.set_ylim([10**(-3), 10**(0)])
    ax.legend(loc='lower right')

    plt.tight_layout()
    plt.show()

In [106]:
recording_names = ['BO11_O5W_Sti_Day1_5uA_m.nwb',
                   'BO11_O5W_Sti_Day1_10uA_m.nwb',
                   'BO11_O5W_Sti_Day1_20uA_m.nwb',
                   'BO11_O5W_Sti_Day1_30uA_m.nwb',
                   'BO11_O5W_Sti_Day1_40uA_m.nwb',
                   'BO11_O5W_Sti_Day1_50uA_m.nwb',
                   'BO11_O5W_Sti_Day1_60uA_m.nwb'
                   ]

amps = ['5ua','10ua','20ua',
        '30ua','40ua','50ua','60ua']
intan_path = 'C:/Users/27707/Documents/jhu_master/LFP_analysis/2024-07-26_DG_3elec/Experimental Set_Final Round_Batch 27/5 Week Old Organoid/Stimulation Day1/'
intan_folders = ['5uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_5uA_233uS_240625_123952/merged/',
                 '10uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_10uA_233uS_240625_125131/merged/',
                 '20uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_20uA_233uS_240625_130048/merged/',
                 '30uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_30uA_233uS_240625_130939/merged/',
                 '40uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_40uA_233uS_240625_131833/merged/',
                 '50uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_50uA_233uS_240625_132735/merged/',
                 '60uA_233.3uS_Biphasic Pulse/BO11_O5W_Stimulation_Day1_60uA_233uS_240625_133646/merged/'
                 ]

In [ ]:
stim_times_array = []
for i in range(len(recording_names)):
    BO7_res = pull_files(intan_folders[i], intan_path)
    stim_names_BO7 = ['STIM_A-000','STIM_A-002','STIM_A-004']
    stim_BO7 = all_intan_data(stim_names_BO7,'stim_data',BO7_res)
    stim_times = stim_BO7.loc[(stim_BO7['STIM_A-000'] != 0) | (stim_BO7['STIM_A-002'] != 0) | (stim_BO7['STIM_A-004'] != 0)].index
    stim_times_array.append(stim_times.to_numpy())

In [ ]:
stim_start_array = []
stim_end_array = []
for stim in stim_times_array:
    stim = np.insert(stim,0,0)
    stim_times_diff = np.diff(stim)
    stim_start = stim[np.where(stim_times_diff > 1)[0]+1]
    stim_end = stim[np.where(stim_times_diff > 1)[0][1:]]
    stim_end = np.concatenate((stim_end,[stim[-1]]))
    print(stim_start)
    print(stim_end)
    stim_start_array.append(stim_start)
    stim_end_array.append(stim_end)

In [ ]:
num_samples = []
num_channels = []
multirecording = []
multirecording_f = []
for i in range(len(recording_names)):
    recording_names = [folder + 'BO11_O5W_Sti_Day1_' + amp.split('u')[0] + 'uA_m.nwb'
                       for folder, amp in zip(intan_folders, amps)]
    rec_names = intan_path + recording_names[i]
    day_recording = load_data(rec_names)
    [num_pre_samples, num_pre_channels] = day_recording.get_traces().shape # Num samples, num channels
    num_samples.append(num_pre_samples)
    num_channels.append(num_pre_channels)
    print('Presample:' + str(num_pre_samples))

    # Case 2: the sorter DOES NOT handle multi-segment objects
    multirecording_1 = day_recording # The `concatenate_recordings()` mimics a mono-segment object that concatenates all segments
    multirecording.append(day_recording) # load your recording using SpikeInterface
    multi_shank = gen_probe()
    multirecording_filter = spre.bandpass_filter(multirecording[i], freq_min=300, freq_max=6000)
    multirecording_f.append(multirecording_filter)

In [ ]:

# Step 1: Calculate RMS-based thresholds for each recording and channel
fs = 30000  # Replace with your actual sampling rate
start_time = 0 # Start at 10 seconds
end_time = 455  # End at 210 seconds
num_samples = fs * (end_time - start_time)  # Calculate the number of samples for 200 seconds

thresholds = []  # Initialize an empty list to store thresholds for each recording

for i in range(len(recording_names)):
    thresholds_per_recording = []  # Store thresholds for each channel in the current recording
    
    for ch in range(3):  # Assuming there are 3 channels
        # Extract the channel data from the 10th second to the 210th second of signal
        channel = multirecording[i].get_traces()[start_time * fs:end_time * fs, ch].astype(np.float32)

        # Calculate the RMS value and determine the threshold
        rms_value = np.sqrt(np.mean(channel**2))
        threshold = rms_value * 5
        thresholds_per_recording.append(threshold)
    
    thresholds.append(thresholds_per_recording)

# Now, use the thresholds in the original artifact detection code
ms_before = [100]
ms_after = [100]
artifact_frames = []

# Original loop for artifact detection
for i in range(len(recording_names)):
    artifact_frames_per_recording = []  # Store artifact frames for each channel in a recording
    
    for ch in range(3):  # Assuming there are 3 channels
        channel = multirecording[i].get_traces()[:, ch].astype(np.float32)
        
        # Use predefined threshold for the current recording and channel
        threshold = thresholds[i][ch]
        print(f"Threshold for recording {i+1}, channel {ch+1}: {threshold}")
        
        # Apply the custom threshold
        stimulation_trigger_frames = np.where(np.abs(channel) > threshold)

        # Remove artifacts using the detected triggers
        if stimulation_trigger_frames[0].size > 0:  # Check if there are any detected triggers
            multirecording_art = spre.remove_artifacts(
                multirecording[i],
                list_triggers=stimulation_trigger_frames[0].tolist(),
                ms_before=ms_before[0],  
                ms_after=ms_after[0]
            )
        else:
            print(f"No artifacts detected for recording {i+1}, channel {ch+1}")
        
        artifact_frames_per_recording.append(stimulation_trigger_frames[0].tolist())

    artifact_frames.append(artifact_frames_per_recording)

In [111]:
from scipy import signal
from spikeinterface.extractors import NumpyRecording
import numpy as np

recording_art = []
recording_lfp = []
recording_mua = []
recording_delta = []
recording_100_200 = []
recording_200_400 = []
recording_gamma = []
artifact_frames_resample = []

# Create notch filters for specific frequencies
#frequencies = [1.4, 9.0, 18.2, 25.0, 27.2, 29.2, 36.4, 37.6, 41.4, 43.8, 45.4, 46.4, 60.0, 180.0, 300.0]
frequencies = [60.0, 180.0, 300.0]
Q_values = [60.0, 180.0, 300.0]  # Define Q values corresponding to each frequency

notch_filters = [signal.iirnotch(f, Q, 30000) for f, Q in zip(frequencies, Q_values)]


for i in range(len(recording_names)):
    rc = []
    for ch in range(3):
        # Remove artifacts for each channel
        recording_artifact_removed = spre.remove_artifacts(
            multirecording[i],
            list_triggers=artifact_frames[i][ch],  # Combine triggers from all channels
            ms_before=ms_before[0], ms_after=ms_after[0]  # Adjust based on your needs
        )
        channel_data_artifact_removed = recording_artifact_removed.get_traces()[:, ch]

        # Append the artifact-removed channel data to rc list
        rc.append(channel_data_artifact_removed)

    # Combine the channels into one array
    combined_data = np.stack(rc, axis=1)  # Stack along the channel axis

    # Create a new RecordingExtractor object with combined data
    sampling_frequency = multirecording[i].get_sampling_frequency()
    combined_recording = NumpyRecording(traces_list=[combined_data], sampling_frequency=sampling_frequency)

    # Append the filtered recording to the list
    recording_art.append(combined_recording)

    # Apply notch filters sequentially to remove noise at specified frequencies
    traces_filtered = recording_art[i].get_traces()
    for b, a in notch_filters:
        traces_filtered = signal.filtfilt(b, a, traces_filtered, axis=0)

    # Create a new RecordingExtractor with the filtered traces
    recording_filtered = NumpyRecording([traces_filtered], sampling_frequency)

    # Process LFP and MUA after filtering
    lfp, mua = downsample_lfp_mua(recording_filtered)

    # Apply filter bank to LFP
    delta, s100_200, s200_400, gamma = filterbank(lfp)
    recording_lfp.append(lfp)
    recording_mua.append(mua)
    recording_delta.append(delta)
    recording_100_200.append(s100_200)
    recording_200_400.append(s200_400)
    recording_gamma.append(gamma)

    # Resample artifact frames for further analysis
    stim_frames = list(map(lambda x: int(x * (1000 / 30000)), np.concatenate(artifact_frames[i])))
    artifact_frames_resample.append(stim_frames)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Extract LFP data from the preprocessed recording_lfp
lfp_data = recording_lfp[3].get_traces()  # Assuming recording_lfp[0] is a RecordingExtractor

# Check the shape of lfp_data to confirm its dimensions
print(f"LFP data shape: {lfp_data.shape}")  # Output the shape to confirm it is 2D (num_samples, num_channels)

# If the LFP data contains multiple channels, select one channel for analysis, e.g., the first channel
lfp_channel_data = lfp_data[:, 2]  # Select the third channel

# Step 1: Compute the power spectral density (PSD) for the 1-20 Hz band
f_20, Pxx = signal.welch(lfp_channel_data, fs=1000, nperseg=400)

# Step 2: Plot the power spectral density to detect noise in the 1-20 Hz band
plt.figure(figsize=(10, 6))
plt.semilogy(f_20, Pxx)
plt.xlim([1, 400])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density [V^2/Hz]')
plt.title('Power Spectral Density (1-20 Hz)')
plt.grid(True)
plt.show()

# Step 3: Use STFT to analyze frequency components in the 1-20 Hz range
f_stft, t_stft, Zxx = signal.stft(lfp_channel_data, fs=1000, nperseg=400)

# Plot the STFT results
plt.figure(figsize=(10, 6))
plt.pcolormesh(t_stft, f_stft, np.abs(Zxx), shading='gouraud')
plt.colorbar(label='Magnitude')
plt.ylim([1, 400])  # Limit the frequency range to 1-20 Hz
plt.xlabel('Time [sec]')
plt.ylabel('Frequency [Hz]')
plt.title('STFT Magnitude (1-20 Hz)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 0  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[1].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    #axs[i].set_xlim([time_axis[0], time_axis[14400000]])
    # axs[i].set_xlim([time_axis[6900000], time_axis[8000000]])
    #axs[i].set_ylim([-0.000025, 0.000025])
axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameter settings
fs = 1000  # Sampling frequency (Hz)
delay_after_stim = 0.6  # Delay time after stimulation (s)
signal_duration = 1.5  # Duration of the signal to extract (s)

# Convert to sample points
delay_samples = int(delay_after_stim * fs)
duration_samples = int(signal_duration * fs)

# Initialize an array to store the average signals
num_channels = raw_data_post.shape[1]
average_signals = []

# Iterate through each channel
for channel_idx in range(num_channels):
    channel_data = raw_data_post[:, channel_idx]
    extracted_signals = []
    
    # Iterate through each stimulation start time
    for stim_start in stim_start_array[1]:  # Assuming stim_start_array corresponds to the first channel
        stim_start_sample = int(stim_start * fs)  # Convert to sample points
        start_sample = stim_start_sample + delay_samples
        end_sample = start_sample + duration_samples
        
        # Ensure valid indices
        if start_sample >= 0 and end_sample <= len(channel_data):
            segment = channel_data[start_sample:end_sample]
            extracted_signals.append(segment)
    
    # Compute the average signal
    if extracted_signals:
        avg_signal = np.mean(extracted_signals, axis=0)
        average_signals.append(avg_signal)

# Time axis (1.5s interval)
time_axis = np.linspace(delay_after_stim, delay_after_stim + signal_duration, duration_samples)

# Plot the average signals
plt.figure(figsize=(10, 6))
for channel_idx, avg_signal in enumerate(average_signals):
    plt.plot(time_axis, avg_signal, label=f'Channel {channel_idx + 1}')
plt.axvline(delay_after_stim, color='red', linestyle='--', label='Stimulus Start + 0.6s')  # Mark post-stimulation delay time
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (V)')
plt.title('Average Signals (1.5s After 0.6s Delay Post-Stimulation)')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft
from sklearn.metrics.pairwise import cosine_similarity

# Parameter settings
fs = 1000  # Sampling frequency (Hz)
delay_after_stim = 0.6  # Delay after stimulation (s)
signal_duration = 1.5  # Duration of the signal to extract (s)
nperseg = 256  # STFT segment length

# Convert parameters to sample points
delay_samples = int(delay_after_stim * fs)
duration_samples = int(signal_duration * fs)

# Initialize storage for signal segments
time_freq_results = []

# Extract signal segments for each channel and perform time-frequency analysis
for channel_idx in range(3):  # Assume we are analyzing only the first three channels
    channel_data = raw_data_post[:, channel_idx]
    channel_segments = []
    for stim_start in stim_start_array[0]:  # Assume stim_start_array corresponds to the first stimulus sequence
        stim_start_sample = int(stim_start * fs)
        start_sample = stim_start_sample + delay_samples
        end_sample = start_sample + duration_samples
        
        # Ensure valid index range
        if start_sample >= 0 and end_sample <= len(channel_data):
            segment = channel_data[start_sample:end_sample]
            # Time-frequency analysis
            f, t, Zxx = stft(segment, fs, nperseg=nperseg)
            channel_segments.append(np.abs(Zxx))  # Extract magnitude spectrum
    time_freq_results.append(channel_segments)

# Compute the average time-frequency distribution for each channel
avg_time_freq_distributions = [np.mean(np.array(segments), axis=0) for segments in time_freq_results]

# Compute cosine similarity
reference = avg_time_freq_distributions[0].flatten()  # Use the first channel as the reference
cosine_similarities = [
    cosine_similarity(reference.reshape(1, -1), dist.flatten().reshape(1, -1))[0, 0]
    for dist in avg_time_freq_distributions
]

# Plot time-frequency distributions
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
for channel_idx, avg_dist in enumerate(avg_time_freq_distributions):
    axs[channel_idx].pcolormesh(t, f, avg_dist, shading='gouraud')
    axs[channel_idx].set_title(f'Channel {channel_idx + 1} - Time-Frequency')
    axs[channel_idx].set_xlabel('Time (s)')
    axs[channel_idx].set_ylabel('Frequency (Hz)')
    axs[channel_idx].set_ylim(1, 50) 
plt.tight_layout()
plt.show()

# Output cosine similarity values
for i, similarity in enumerate(cosine_similarities):
    print(f'Cosine similarity of Channel {i+1} to Channel 1: {similarity:.4f}')




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the parameters
fs = 1000  # Sampling frequency (30kHz)
time_window = 0  # 5 minutes in seconds
start_sample = int(time_window * fs)  # Starting sample at the 5-minute mark

# Extract the data from 5 minutes to the end
raw_data_post = recording_lfp[6].get_traces()[start_sample:, :]

# Calculate the time axis for the plot
time_axis = np.arange(start_sample, start_sample + raw_data_post.shape[0]) / fs

# Plot the raw signal for each channel
num_channels = raw_data_post.shape[1]
fig, axs = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

for i in range(num_channels):
    axs[i].plot(time_axis, raw_data_post[:, i])
    axs[i].set_title(f'Channel {i+1}')
    axs[i].set_ylabel('Amplitude (V)')
    axs[i].set_xlim([time_axis[0], time_axis[480000]])

axs[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Calculate and print the average amplitude for each channel
average_amplitudes = np.mean(np.abs(raw_data_post), axis=0)
for i, avg_amp in enumerate(average_amplitudes):
    print(f'Average amplitude for Channel {i+1}: {avg_amp:.6f} V')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

fs_stim = 1000  # Sampling frequency for stimulus data


fig, axs = plt.subplots(5, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[0][:4]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[0].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg31 = coherence_sum3 / 4

# Plot the average coherence in the last row
for j in range(3):
    axs[4, j].semilogy(f3, coherence_avg31[:, j])
    axs[4, j].set_title(f'{titles[j]} (Average)')
    axs[4, j].set_xlabel('Frequency [Hz]')
    axs[4, j].set_ylabel('Coherence')
    axs[4, j].set_xlim([5, 400])
    axs[4, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[1][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[1].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg32 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg32[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[2][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[2].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg33 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg33[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[3][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[3].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg34 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg34[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[4][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[4].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg35 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg35[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[5][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[5].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg36 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg36[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(6, 3, sharex=True, sharey=True, dpi=400, figsize=(16, 18))  # Six rows, three columns
titles = ['North-East', 'North-West', 'East-West']
ind1 = [0, 0, 1]
ind2 = [1, 2, 2]

coherence_sum3 = np.zeros((int(nperseg/2+1), 3))  # Initialize an array to store sum of coherence values

# Step 2: Calculate Post-Stimulus Coherence (100 ms after each of the first five pulses)
for i, pulse_time in enumerate(stim_start_array[6][:5]):  # Iterate over the first 5 pulse times
    segment = recording_lfp[6].get_traces()[int(pulse_time * fs_stim + time_buffer * fs_stim):int((pulse_time + time_buffer + time_period) * fs_stim), :]
    
    for j in range(3):
        x_post = segment[:, ind1[j]]
        y_post = segment[:, ind2[j]]
        f3, Cxy_post3 = signal.coherence(x_post, y_post, fs_stim, nperseg=nperseg, noverlap=overlap)
        
        coherence_sum3[:, j] += Cxy_post3  # Accumulate coherence values
        
        axs[i, j].semilogy(f3, Cxy_post3)
        axs[i, j].set_title(f'{titles[j]} (Pulse {i+1})')
        axs[i, j].set_xlabel('Frequency [Hz]')
        axs[i, j].set_ylabel('Coherence')
        axs[i, j].set_xlim([5, 400])
        axs[i, j].set_ylim([10**(-3), 10**(0)])

# Calculate the average coherence
coherence_avg37 = coherence_sum3 / 5

# Plot the average coherence in the last row
for j in range(3):
    axs[5, j].semilogy(f3, coherence_avg37[:, j])
    axs[5, j].set_title(f'{titles[j]} (Average)')
    axs[5, j].set_xlabel('Frequency [Hz]')
    axs[5, j].set_ylabel('Coherence')
    axs[5, j].set_xlim([5, 400])
    axs[5, j].set_ylim([10**(-3), 10**(0)])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize an array to store the average coherence values for all recordings
all_coherence_avg3 = np.zeros((int(nperseg / 2 + 1), 7, 3))  # 7 recordings, 3 coherence pairs

# Assign the precomputed coherence averages to the all_coherence_avg array
all_coherence_avg3[:, 0, :] = coherence_avg31
all_coherence_avg3[:, 1, :] = coherence_avg32
all_coherence_avg3[:, 2, :] = coherence_avg33
all_coherence_avg3[:, 3, :] = coherence_avg34
all_coherence_avg3[:, 4, :] = coherence_avg35
all_coherence_avg3[:, 5, :] = coherence_avg36
all_coherence_avg3[:, 6, :] = coherence_avg37

# Plot average coherence curves in separate figures for each pair
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
labels = [f'{i}' for i in amps]

for j in range(3):  # Loop over each of the three coherence pairs
    fig, ax = plt.subplots(dpi=400, figsize=(10, 8))

    for k in range(7):
        ax.semilogy(f3, all_coherence_avg3[:, k, j], label=labels[k], color=colors[k])

    ax.set_title(f'Average Coherence - {titles[j]}')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Coherence')
    ax.set_xlim([1, 50])
    ax.set_ylim([10**(-3), 10**(0)])
    ax.legend(loc='lower right')

    plt.tight_layout()
    plt.show()

In [ ]:
from scipy.interpolate import interp1d

# Initialize arrays for interpolated coherence data
interp_coherence1 = np.zeros((f.size, 7, 3))
interp_coherence2 = np.zeros((f.size, 7, 3))
interp_coherence3 = np.zeros((f.size, 7, 3))
interp_coherence4 = np.zeros((f.size, 7, 3))

# Perform interpolation for two datasets
for i in range(7):  # 7 recordings
    for j in range(3):  # Each recording has 3 coherence pairs
            interp_func1 = interp1d(f1, all_coherence_avg1[:, i, j], kind='linear', bounds_error=False, fill_value="extrapolate")
            interp_func2 = interp1d(f, all_coherence_avg[:, i, j], kind='linear', bounds_error=False, fill_value="extrapolate")
            interp_func3 = interp1d(f2, all_coherence_avg2[:, i, j], kind='linear', bounds_error=False, fill_value="extrapolate")
            interp_func4 = interp1d(f3, all_coherence_avg3[:, i, j], kind='linear', bounds_error=False, fill_value="extrapolate")

            # Apply interpolation to the common frequency range
            interp_coherence1[:, i, j] = interp_func1(f)
            interp_coherence2[:, i, j] = interp_func2(f)
            interp_coherence3[:, i, j] = interp_func3(f)
            interp_coherence4[:, i, j] = interp_func4(f)

# Compute the average interpolated coherence data
average_coherence_final = (interp_coherence1 + interp_coherence2 + interp_coherence3 + interp_coherence4) / 4

plt.rcParams['font.family'] = 'Arial'

# Plotting
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Colors
titles = ['North-East', 'North-West', 'East-West']
amps = [5, 10, 20, 30, 40, 50, 60]  
xticks_values = [10, 20, 30, 40, 50]  # Manually set x-axis ticks

for j in range(3):
    fig, ax = plt.subplots(dpi=400, figsize=(5, 5))
    for k in range(7):
        ax.semilogy(f, average_coherence_final[:, k, j], label=f'{amps[k]}µA', color=colors[k])

    ax.set_title(f'{titles[j]}', fontsize=27, fontweight='bold')
    ax.set_xlabel('Frequency (Hz)', fontsize=25, fontweight='bold')
    ax.set_ylabel('Coherence', fontsize=25, fontweight='bold')
    ax.set_xlim([1, 50])
    ax.set_xticks(xticks_values)  # Set x-axis ticks
    ax.set_xticklabels([str(tick) for tick in xticks_values])  # Ensure labels are strings
    ax.set_ylim([10**(-3), 10**(0)])
    ax.tick_params(axis='both', which='major', labelsize=23, width=2, length=5) 
    ax.legend(loc='lower right', fontsize=14.5)
    plt.tight_layout()
    plt.show()
